In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

# Carregue seus dados em um dataframe (supondo que o nome do dataframe seja "dados")
dados = pd.read_csv("data/trainWithFlight.csv")
del dados["HoraData"],dados["HoraDataDest"]

FileNotFoundError: [Errno 2] No such file or directory: 'data/trainWithFlight.csv'

In [4]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Divide the data into features (X) and target column (y)
X = dados.drop(["target", "flightid"], axis=1)
y = dados["target"]
# Find columns with 4 or fewer unique values in X
cols_to_transform = [col for col in X.columns if X[col].nunique() <= 4]

# Apply get_dummies on these columns
X = pd.get_dummies(X, columns=cols_to_transform)
# Divide the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# A good practice is standardizing the features for better model performance
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Create a LightGBM Dataset object for better performance
d_train = lgb.Dataset(X_train, label=y_train)

# Define the parameters of the LightGBM model
params = {
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "bagging_fraction": 0.7,
    "feature_fraction": 0.7,
    "bagging_frequency": 5,
    "bagging_seed": 42,
    "verbosity": 1
}

# Train the LightGBM model with early stopping
#model = lgb.train(params, d_train, num_boost_round=10000, valid_sets=[d_train], callbacks=(
#            [lgb.early_stopping(stopping_rounds=100)]))
#
## Make predictions on the test set using the best iteration
#y_pred = model.predict(X_test, num_iteration=model.best_iteration)
#
## Calculate the root mean square error (RMSE) in the predictions
#rmse = mean_squared_error(y_test, y_pred, squared=False)
#print(f"RMSE: {rmse}")


NameError: name 'dados' is not defined

In [ ]:
from bayes_opt import BayesianOptimization

# Define the parameter search space using Bayesian Optimization
def lgb_optimization(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
    
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': int(num_leaves),
        'feature_fraction': float(feature_fraction),
        'bagging_fraction': float(bagging_fraction),
        'max_depth': int(max_depth),
        'lambda_l1': float(lambda_l1),
        'lambda_l2': float(lambda_l2),
        'min_split_gain': float(min_split_gain),
        'min_child_weight': float(min_child_weight),
        'verbosity': -1
    }
   
    model = lgb.train(params, d_train, num_boost_round=10000, valid_sets=[d_train], early_stopping_rounds=100, verbose_eval=False)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    
    score = mean_squared_error(y_test.values, y_pred, squared=False)
    
    return -score

optimizer = BayesianOptimization(
    lgb_optimization,
    {
        'num_leaves': (5, 130),
        'feature_fraction': (0.1, 1.0),
        'bagging_fraction': (0.1, 1.0),
        'max_depth': (-1, 150),
        'lambda_l1': (0, 5),
        'lambda_l2': (0, 5),
        'min_split_gain': (0.001, 0.1),
        'min_child_weight': (5, 50)
    }
)

# Perform the Bayesian Optimization
optimizer.maximize(init_points=10, n_iter=30)
